In [10]:
#Importing all necessery libraries
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.tensorboard import SummaryWriter


In [11]:
# Defining here the transformations for dataset
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize images to a fixed size
    transforms.ToTensor(),        # Convert images to PyTorch tensors
    transforms.Normalize((0.5,), (0.5,))  # Normalize pixel values between -1 and 1
])


# Here loading dataset from the directory
train_data = ImageFolder(root='data/data/train', transform=transform)
test_data = ImageFolder(root='data/data/test', transform=transform)


# Creating here the DataLoaders for batching
train_loader = DataLoader(dataset=train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=32, shuffle=False)


In [12]:
# Checking the number of images in the training and test datasets
print(f'Number of training samples: {len(train_data)}')
print(f'Number of test samples: {len(test_data)}')

# Checking the class labels in the training data (should match your 7 folders)
print(f'Class labels (train): {train_data.classes}')


Number of training samples: 3467
Number of test samples: 8
Class labels (train): ['accessories', 'jackets', 'jeans', 'knitwear', 'shirts', 'shoes', 'shorts', 'tees']


In [13]:
#Defining the MLP model class:

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        
        # Define the architecture with fully connected layers
        self.fc1 = nn.Linear(input_size, hidden_size)  # First hidden layer
        self.relu1 = nn.ReLU()  # Activation function
        self.fc2 = nn.Linear(hidden_size, hidden_size)  # Second hidden layer
        self.relu2 = nn.ReLU()  # Activation function
        self.fc3 = nn.Linear(hidden_size, output_size)  # Output layer
    
    def forward(self, x):
        # Flatten the input image
        x = x.view(x.size(0), -1)
        
        # Forward pass through the layers
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x


In [14]:
#Define the hyperparameters 
input_size = 64 * 64 * 3  # Image size (64x64 with 3 color channels)
hidden_size = 128  # Number of neurons in hidden layers
output_size = 8  # Number of classes (for your 8 categories)
learning_rate = 0.001
num_epochs = 10


Initializing the Model:

In [15]:

model = MLP(input_size=input_size, hidden_size=hidden_size, output_size=output_size)


criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


Training the Model:

In [16]:
writer = SummaryWriter('runs/mlp_experiment')
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        
        
        optimizer.zero_grad()
        
       
        outputs = model(images)
        loss = criterion(outputs, labels)
        
       
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}')
            writer.add_scalar('Training Loss', running_loss / 100, epoch * len(train_loader) + i)
            running_loss = 0.0


writer.close()


Epoch [1/10], Step [100/109], Loss: 0.8418
Epoch [2/10], Step [100/109], Loss: 0.5746
Epoch [3/10], Step [100/109], Loss: 0.4737
Epoch [4/10], Step [100/109], Loss: 0.4310
Epoch [5/10], Step [100/109], Loss: 0.3908
Epoch [6/10], Step [100/109], Loss: 0.3450
Epoch [7/10], Step [100/109], Loss: 0.2962
Epoch [8/10], Step [100/109], Loss: 0.3086
Epoch [9/10], Step [100/109], Loss: 0.2739
Epoch [10/10], Step [100/109], Loss: 0.2227


Evaluating the model:

In [18]:
model.eval()

correct = 0
total = 0

with torch.no_grad(): 
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()


accuracy = (correct / total) * 100
print(f'Test Accuracy: {accuracy:.2f}%')

# Saving the trained model
torch.save(model.state_dict(), 'mlp_model.pth')


Test Accuracy: 25.00%
